# **Wstęp**


## Cel zadania

W projekcie 4 mieliśmy do rozwiązania pięć zadań podanych za pomocą poleceń SQL. Korzystając z pakietu `pandas` otrzymywaliśmy konkretne ramki danych, które porównywaliśmy z tymi, które zostały wywołane przy użyciu funckji `pandas.read_sql_query`.

## Pakiety

Najpierw dodamy odpowiednie pakiety i ramki danych: 

In [1]:
import pandas as pd
import numpy as np

import os, os.path
import sqlite3
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
Tags = pd.read_csv("travel_stackexchange_com/Tags.csv.gz", compression = 'gzip')
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv.gz", compression = 'gzip')
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz", compression = 'gzip')
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz", compression = 'gzip')
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz", compression = 'gzip')
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz", compression = 'gzip')

# Baza danych

Poniżej tworzymy tymczasową bazę danych:

In [3]:
baza = os.path.join('przyklad.db') # sciezka dostępu do bazy danych
if os.path.isfile(baza):           # jesli baza już istneje... 
    os.remove(baza)                # ...usuniemy ja
    
conn = sqlite3.connect(baza)       # połączenie do bazy danych
Badges.to_sql("Badges", conn) 
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

953600

# Zadanie 1

In [4]:
# Zad. 1

# a) SQL

wynik11 = pd.read_sql_query(""" SELECT Count, TagName
FROM Tags
WHERE Count > 1000
ORDER BY Count DESC""", conn)


# b) pandas

wynik12 = Tags[['Count', 'TagName']]
wynik12 = wynik12[wynik12.Count > 1000]
wynik12 = wynik12.sort_values(by = ['Count'], ascending = False)
wynik12 = wynik12.reset_index(drop = True)

display(wynik12)

,Count,TagName
0,9470,visas
1,5119,usa
2,4601,uk
3,4460,air-travel
4,3503,customs-and-immigration
5,3296,schengen
6,2058,transit
7,1695,passports
8,1665,indian-citizens
9,1517,trains


# Zadanie 2

In [5]:
# Zad. 2

# a) SQL

wynik21 = pd.read_sql_query("""SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10""", conn)


# b) pandas

wynik22 = Users.merge(Posts, how = 'inner', left_on = ['Id'], right_on = ['OwnerUserId'])
wynik22 = wynik22[['Id_x', 'Location']]
wynik22 = wynik22[wynik22.Location != '']
wynik22 = wynik22.groupby(['Location'], as_index=False)['Location'].size()
wynik22.rename(columns = {'Location': 'Location', 'size': 'Count'}, inplace = True)
wynik22 = wynik22.sort_values(by = ['Count'], ascending = False)
wynik22 = wynik22.reset_index(drop = True)
wynik22 = wynik22.head(10)

display(wynik22)

,Location,Count
0,"Christchurch, New Zealand",2765
1,"New York, NY",1788
2,"London, United Kingdom",1708
3,UK,1590
4,"Sunshine Coast QLD, Australia",1550
5,Australia,1183
6,"Vancouver, Canada",967
7,Netherlands,935
8,on the server farm,924
9,Pennsylvania,921


# Zadanie 3

In [6]:
# Zad. 3

# a) SQL

wynik31 = pd.read_sql_query("""SELECT Year, SUM(Number) AS TotalNumber
FROM (
SELECT
Name,
COUNT(*) AS Number,
STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
WHERE Class = 1
GROUP BY Name, Year
)
GROUP BY Year
ORDER BY TotalNumber""", conn)


# b) pandas

wynik32 = Badges[Badges.Class == 1]
wynik32 = wynik32[['Name', 'Date']]
wynik32['Date'] = pd.to_datetime(wynik32.Date)
wynik32['Date1'] = wynik32['Date'].dt.strftime('%Y')
wynik32 = wynik32.groupby(['Date1'], as_index = False)['Date1'].size()
wynik32.rename(columns = {'Date1': 'Year', 'size': 'TotalNumber'}, inplace = True)
wynik32 = wynik32.sort_values(by = ['TotalNumber'], ascending = True)
wynik32 = wynik32.reset_index(drop = True)


display(wynik32)

,Year,TotalNumber
0,2011,16
1,2012,23
2,2013,66
3,2021,153
4,2014,197
5,2020,265
6,2015,328
7,2016,509
8,2017,552
9,2018,697


# Zadanie 4

In [7]:
# Zad. 4

# a) SQL

wynik41 = pd.read_sql_query("""SELECT
        Users.AccountId,
        Users.DisplayName,
        Users.Location,
        AVG(PostAuth.AnswersCount) as AverageAnswersCount
        FROM
        (
          SELECT
          AnsCount.AnswersCount,
          Posts.Id,
          Posts.OwnerUserId
          FROM (
            SELECT Posts.ParentId, COUNT(*) AS AnswersCount
            FROM Posts
            WHERE Posts.PostTypeId = 2
            GROUP BY Posts.ParentId
          ) AS AnsCount
          JOIN Posts ON Posts.Id = AnsCount.ParentId
        ) AS PostAuth
        JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
        GROUP BY OwnerUserId
        ORDER BY AverageAnswersCount DESC, AccountId ASC
        LIMIT 10""", conn)


# b) pandas

AnsCount = Posts[Posts.PostTypeId == 2]
AnsCount = AnsCount.groupby(['ParentId'], as_index = False)['ParentId'].size()
AnsCount.rename(columns = {'ParentId': 'ParentId', 'size': 'AnswersCount'}, inplace = True)

PostAuth = Posts.merge(AnsCount, how = 'inner', left_on = ['Id'], right_on = ['ParentId'])
PostAuth = PostAuth[['AnswersCount', 'Id', 'OwnerUserId']]

wynik42 = Users.merge(PostAuth, how = 'inner', left_on = ['AccountId'], right_on = ['OwnerUserId'])
wynik42 = wynik42[['AccountId', 'DisplayName', 'Location', 'AnswersCount']]

avg = wynik42.groupby(['AccountId'], as_index = False)['AnswersCount'].mean()

wynik42 = wynik42[['AccountId', 'DisplayName', 'Location']]
wynik42 = avg.merge(wynik42, how = 'inner', left_on = ['AccountId'], right_on = ['AccountId'])
wynik42.rename(columns = {'AnswersCount': 'AverageAnswersCount'}, inplace = True)
wynik42 = wynik42[['AccountId', 'DisplayName', 'Location', 'AverageAnswersCount']]
wynik42 = wynik42.sort_values(['AverageAnswersCount', 'AccountId'], ascending = [False, True])
wynik42 = wynik42.head(10)
wynik42 = wynik42.reset_index(drop = True)


display(wynik42)

,AccountId,DisplayName,Location,AverageAnswersCount
0,280.0,csmba,"San Francisco, CA",11.0
1,40811.0,vocaro,"San Jose, CA",11.0
2,204.0,Josh,Australia,10.0
3,44093.0,Emma Arbogast,"Salem, OR",10.0
4,11758.0,rvarcher,"Oklahoma City, OK",9.0
5,19588.0,JD Isaacks,"Atlanta, GA",8.0
6,20473.0,Jeremy Boyd,"Houston, TX",8.0
7,42364.0,Petrogad,NaN,8.0
8,54571.0,Christian,NaN,8.0
9,79346.0,Thomas Matthews,California,8.0


# Zadanie 5

In [8]:
# Zad. 5

# a) SQL

wynik51 = pd.read_sql_query("""SELECT Posts.Title, Posts.Id,
        STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date,
        VotesByAge.Votes
        FROM Posts
        JOIN (
          SELECT
          PostId,
          MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
          MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
          SUM(Total) AS Votes
          FROM (
            SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
            WHEN '2021' THEN 'new'
            WHEN '2020' THEN 'new'
            ELSE 'old'
            END VoteDate,
            COUNT(*) AS Total
            FROM Votes
            WHERE VoteTypeId IN (1, 2, 5)
            GROUP BY PostId, VoteDate
          ) AS VotesDates
          GROUP BY VotesDates.PostId
          HAVING NewVotes > OldVotes
        ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
        WHERE Title NOT IN ('')
        ORDER BY Votes DESC
        LIMIT 10""", conn)


# b) pandas

VotesDates = Votes[(Votes.VoteTypeId == 1) | (Votes.VoteTypeId == 2)  | (Votes.VoteTypeId == 5)]
VotesDates['CreationDate'] = pd.to_datetime(VotesDates['CreationDate'])
VotesDates['CreationDate1'] = VotesDates['CreationDate'].dt.strftime('%Y')
VotesDates = VotesDates[['PostId', 'CreationDate1']]
VotesDates.rename(columns = {'CreationDate1': 'CreationDate'}, inplace = True)
VotesDates.loc[VotesDates['CreationDate'] == '2021', 'CreationDate'] = 'new'
VotesDates.loc[VotesDates['CreationDate'] == '2020', 'CreationDate'] = 'new'
VotesDates.loc[VotesDates['CreationDate'] != 'new', 'CreationDate'] = 'old'
VotesDates = VotesDates.groupby(['PostId', 'CreationDate'], as_index = False)['CreationDate'].size()
df = pd.DataFrame(VotesDates)
VotesDates = df
VotesDates = VotesDates.sort_values(by = ['PostId'], ascending = True)
VotesDates.columns = ['PostId', 'VoteDate', 'Total']

VotesByAge1 = VotesDates[VotesDates.VoteDate == 'new']
VotesByAge2 = VotesDates[VotesDates.VoteDate == 'old']
data = {'PostId':  np.arange(1, 151675, 1),
        'VoteDate': ['old' for i in range(1, 151675, 1)],
         'Total': [0 for j in range(1, 151675, 1)]
        }
pomdf = pd.DataFrame(data)

VotesByAge2 = VotesByAge2.merge(pomdf, how = 'outer', left_on = ['PostId'], right_on = ['PostId'])
VotesByAge2 = VotesByAge2[['PostId', 'Total_x', 'VoteDate_y', 'Total_y']]
VotesByAge2 = VotesByAge2.replace(np.nan, 0)

VotesByAge2['Total'] = VotesByAge2['Total_x'] + VotesByAge2['Total_y']
VotesByAge2 = VotesByAge2[['PostId', 'VoteDate_y', 'Total']]


VotesByAge = VotesByAge1.merge(VotesByAge2, how = 'outer', left_on = ['PostId'], right_on = ['PostId'])
del VotesByAge["VoteDate"]
del VotesByAge["VoteDate_y"]
VotesByAge = VotesByAge.replace(np.nan, 0)
VotesByAge.rename(columns = {'Total_x': 'NewVotes', 'Total_y': 'OldVotes'}, inplace = True)
VotesByAge = VotesByAge[VotesByAge["NewVotes"] > VotesByAge['OldVotes']]

VotesByAge['Votes'] = VotesByAge['NewVotes'] + VotesByAge['OldVotes']
VotesByAge = VotesByAge.reset_index(drop = True)

wynik52 = Posts.merge(VotesByAge, how = 'inner', left_on = ['Id'], right_on = ['PostId'])
wynik52 = wynik52[["Title", "Id", "CreationDate", "Votes"]]
wynik52 = wynik52.dropna()
wynik52 = wynik52[wynik52.Title != '']
wynik52['CreationDate'] = pd.to_datetime(wynik52.CreationDate)
wynik52['CreationDate1'] = wynik52['CreationDate'].dt.strftime('%Y-%m-%d')
wynik52 = wynik52[["Title", "Id", "CreationDate1", "Votes"]]
wynik52.rename(columns = {'CreationDate1': 'Date'}, inplace = True)
wynik52 = wynik52.sort_values(by = ['Votes', 'Id'], ascending = [False, True])
wynik52 = wynik52.head(10)
wynik52['Votes'] = wynik52['Votes'].astype(np.int64)
wynik52 = wynik52.reset_index(drop = True)


display(wynik52)

,Title,Id,Date,Votes
0,What's the longest distance that can be travel...,151994,2020-01-09,140
1,Considerations for very fragile and expensive ...,157138,2020-05-10,108
2,What are these chair-like things in hotels?,153605,2020-02-10,94
3,What to do if I overstayed my e-visa for Saint...,152418,2020-01-18,84
4,What are the hidden fees of a cruise?,152178,2020-01-13,80
5,What happens when a town is under quarantine a...,152677,2020-01-23,76
6,I've been warned to leave the US within 10 day...,156945,2020-04-28,72
7,"When an individual enters the United States, c...",159298,2020-09-06,68
8,Is Seiryu Miharashi Station the only train sta...,162386,2021-02-04,64
9,Positive drug test in Singapore: How long do I...,163264,2021-04-08,64


# Porównanie wyników

Musimy zagwarantować, że zwracane wyniki w każdym przypadku są tożsame.

In [9]:
print(wynik11.equals(wynik12))
print(wynik21.equals(wynik22))
print(wynik31.equals(wynik32))
print(wynik41.equals(wynik42))
print(wynik51.equals(wynik52))

True
True
True
True
True


# Sprawdzenie typów danych 

Musimy zagwarantować, że zwracane wyniki są typu `pandas.Dataframe`.

In [10]:
print(type(wynik11), type(wynik12))
print(type(wynik21), type(wynik22))
print(type(wynik31), type(wynik32))
print(type(wynik41), type(wynik42))
print(type(wynik51), type(wynik52))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


Zamykamy połączenie z bazą danych.

In [11]:
conn.close()

# Wnioski

1. Pakiet `pandas` to wygodne narzędzie do obróbki danych.
2. Rozwiązania za pomocą `pandas.read_sql_query` są pomocne przy porównywaniu wyników.